# 2D Distance Finder
**NOTE:** Currently this tool does not work on EIT, it can only be ran locally! A MATLAB version of this tool is available in the `MATLAB` folder.

This tool allows you to measure the distance between two points on a 2D image. Upload a folder containing the image stack you want to analyze, and click on the image to measure the distance between two points. The distance will be displayed in pixels and millimeters.

<center>
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/2D_distance_demo.gif" alt="2D Distance Demo" width="50%" />
</center>


# Upload the Image Stack
Select the folder containing the image stack you want to analyze. The image stack should be in a folder containing the image files (e.g. MRI_4, CT_1, etc.).

In [1]:
!pip install ipywidgets ipython > /dev/null 2>&1
!pip install jupyter_contrib_nbextensions

!jupyter nbconvert --to notebook --nbformat 4 2D_distance_finder.ipynb --output 2D_distance_finder_converted.ipynb
import os
import ipywidgets as widgets
from IPython.display import display

# Initialize the current path
current_path = os.getcwd()  # This will hold the current directory path

# Create widgets
path_display = widgets.Text(
    value=current_path,
    description='Path:',
    layout=widgets.Layout(width='800px')
)

output = widgets.Output()

def display_directories(path):
    """Display only directories in the given path as clickable rectangles."""
    items = [item for item in os.listdir(path) if os.path.isdir(os.path.join(path, item))]
    buttons = []

    for item in items:
        item_path = os.path.join(path, item)
        button = widgets.Button(
            description=item,
            layout=widgets.Layout(width='auto', height='30px')
        )
        button.style.button_color = '#007bff'
        button.on_click(lambda b, p=item_path: handle_directory_click(p))
        buttons.append(button)

    return widgets.VBox(buttons)

def count_tif_files(path):
    """Count the number of .tif files in the given directory."""
    return len([f for f in os.listdir(path) if f.endswith('.tif')])

def handle_directory_click(directory_path):
    """Handle clicks on directories."""
    global current_path  # Make current_path accessible globally
    current_path = directory_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

    # Check for .tif files and display a warning if there are less than two
    tif_count = count_tif_files(directory_path)
    output.clear_output()
    with output:
        if tif_count <= 1:
            print(f"Warning: The directory '{os.path.basename(directory_path)}' contains {tif_count} .tif file(s).")
        else:
            print(f"The directory '{os.path.basename(directory_path)}' contains enough ({tif_count}) .tif file(s).")

def navigate_to_parent_directory(_):
    """Navigate to the parent directory."""
    global current_path  # Make current_path accessible globally
    parent_path = os.path.dirname(current_path)
    current_path = parent_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

def refresh_directory_view(path):
    """Refresh the directory view."""
    directory_view.children = [display_directories(path)]

# Button to go to the parent directory
parent_button = widgets.Button(
    description='Go to Parent Directory',
    layout=widgets.Layout(width='auto', height='30px')
)
parent_button.style.button_color = '#007bff'
parent_button.on_click(navigate_to_parent_directory)

directory_view = widgets.VBox(children=[display_directories(current_path)])

navigation_box = widgets.VBox([path_display, parent_button, directory_view, output])
try:
    display(navigation_box)
except Exception as e:
    print("Widget display failed. Please enter the directory path manually.")
    current_path = input("Enter the directory path: ")
    print(f"Using directory: {current_path}")


# If the current path stops updating, run this cell again


Traceback (most recent call last):
  File "/Users/alexandergadin/miniconda3/lib/python3.11/site-packages/nbconvert/filters/markdown_mistune.py", line 25, in <module>
    from mistune import (  # type:ignore[attr-defined]
  File "/Users/alexandergadin/miniconda3/lib/python3.11/site-packages/mistune/__init__.py", line 4, in <module>
    from .renderers import AstRenderer, HTMLRenderer
ImportError: cannot import name 'AstRenderer' from 'mistune.renderers' (/Users/alexandergadin/miniconda3/lib/python3.11/site-packages/mistune/renderers/__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/alexandergadin/miniconda3/bin/jupyter-nbconvert", line 7, in <module>
    from nbconvert.nbconvertapp import main
  File "/Users/alexandergadin/miniconda3/lib/python3.11/site-packages/nbconvert/__init__.py", line 7, in <module>
    from .exporters import (
  File "/Users/alexandergadin/miniconda3/lib/python3.11/site-packages/nb

# Start the Viewer
Run the cell below to start the viewer. A window will appear with a scroll bar to navigate through the images. Click on the image to measure the distance between two points. The distance will be displayed in pixels and millimeters. There can only be one line present at a time and its distance will be printed to the console.

In [2]:
import sys
import os
import math
import datetime
import numpy as np
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QSlider, QFileDialog
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen
from PyQt5.QtCore import Qt, QPoint
from PIL import Image


class ImageMeasureApp(QMainWindow):
    def __init__(self, folder_path):
        super().__init__()
        self.folder_path = folder_path
        self.tif_files = self.get_tif_files(folder_path)
        self.current_index = 0
        self.init_ui()

    def get_tif_files(self, folder_path):
        """Retrieve all .tif files in the specified folder."""
        return sorted([f for f in os.listdir(folder_path) if f.endswith(".tif")])

    def init_ui(self):
        if not self.tif_files:
            print(f"No .tif files found in {self.folder_path}")
            sys.exit()

        self.load_image(self.tif_files[self.current_index])

        self.label = QLabel(self)
        self.label.setPixmap(self.pixmap)
        self.label.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        self.label.setScaledContents(True)
        self.label.mousePressEvent = self.get_point

        self.points = []

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.tif_files) - 1)
        self.slider.setValue(self.current_index)
        self.slider.valueChanged.connect(self.update_image)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.slider)

        container = QWidget()
        container.setLayout(self.layout)
        self.setCentralWidget(container)

        self.setWindowTitle("Measure Distance on Image with File Navigation")
        self.resize(800, 800)
        self.show()

    def load_image(self, filename):
        """Load the image and prepare the pixmap."""
        image_path = os.path.join(self.folder_path, filename)
        self.image = Image.open(image_path)
        self.image_array = np.array(self.image)


        height, width = self.image_array.shape
        image_qt = QImage(
            self.image_array.data,
            width,
            height,
            self.image_array.strides[0],
            QImage.Format_Grayscale8
        )
        self.pixmap = QPixmap.fromImage(image_qt)
        self.original_pixmap = self.pixmap.copy()

    def update_image(self, value):
        """Update the displayed image when the slider value changes."""
        self.current_index = value
        self.load_image(self.tif_files[self.current_index])
        self.label.setPixmap(self.pixmap)
        self.points.clear()

    def get_point(self, event):
        """Capture click points on the image and calculate distance."""
        x = event.pos().x()
        y = event.pos().y()
        self.points.append(QPoint(x, y))

        if len(self.points) == 2:
            self.pixmap = self.original_pixmap.copy()
            painter = QPainter(self.pixmap)
            pen = QPen(Qt.blue, 3)
            painter.setPen(pen)
            painter.drawLine(self.points[0], self.points[1])
            painter.end()

            self.label.setPixmap(self.pixmap)
            x1, y1 = self.points[0].x(), self.points[0].y()
            x2, y2 = self.points[1].x(), self.points[1].y()
            pixel_distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            mm_distance = pixel_distance * (10 / 17.52)

            print(f"Distance: {pixel_distance:.2f} pixels ({mm_distance:.2f} mm)")
            self.points.clear()


# Main execution
if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = ImageMeasureApp(current_path)
    now = datetime.datetime.now()
    print(f"{now} - The cell has been completed")
    sys.exit(app.exec_())


2025-01-23 11:52:03.947414 - The cell has been completed


2025-01-23 11:52:04.031 python[1821:25941] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-23 11:52:04.031 python[1821:25941] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Distance: 114.04 pixels (65.09 mm)
Distance: 96.88 pixels (55.29 mm)


SystemExit: 0

/Users/alexandergadin/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
